In [1]:
import sys
import os
import numpy as np
from opentrons import robot, instruments, labware
import itertools

#Original Directory
og_dir = os.getcwd()

# #Change your_path to your system's ../ot2protocols. You may or may not need r in front. 
# #I couldn't get this to work otherwise. I had to modify ot2protocols.__init__:
#from .ot2protocols import* -> from ot2protocols import*
your_path = r'C:\Users\lacho\OneDrive - UW\Pozzo-RG-OT2\OT2Protocols\ot2protocol'
os.chdir(your_path)
from Components import *
from A1Stock import *
from Mineralization_surfactant_oil_sample_class import *


#Return to original directory
os.chdir(og_dir)



C:\Users\lacho\.opentrons\deck_calibration.json not found. Loading defaults
C:\Users\lacho\.opentrons\robot_settings.json not found. Loading defaults


Loading json containers...
Json container file load complete, listing database
Found 0 containers to add. Starting migration...
Database migration complete!


In [29]:


def dict_product(dicts):
    """
    Input a dictionary of key:component range array. where key is the role of the component in the sample, 
    e.g. 'stabilizer1':sodiumdodecylsulfate_range_list
    
    Generate a cartesian product in the form of a list of dictionaries containing all possible combinations. 
    >>> ]list(dict_product(dict(number=[1,2], character='ab')))
    [{'character': 'a', 'number': 1},
     {'character': 'a', 'number': 2},
     {'character': 'b', 'number': 1},
     {'character': 'b', 'number': 2}]
    """
    return (dict(zip(dicts, x)) for x in itertools.product(*dicts.values()))

def make_sample_list(sample_volume, components_dict, volf_dict_list, molarity_dict_list, stock_dict ):
    #All samples which are spanned by the initialization parameters. 
    raw_sample_object_list = []
    #Samples which can be made with this experimental setup
    safe_sample_object_list = []
    #Samples which cannot be made linked with associated reason tuple -> (sample_object, "string reason").  
    rejected_sample_object_tuple_list = []
    
    #Iterate through all possible volfs and molarities, and create samples with those target values
    #Add checks for whether volume of sample is too small too move, or if other issues arise.
    for volf_dict in volf_dict_list:
        for molarity_dict in molarity_dict_list:
            sample_ok = True
            new_sample = Oil_Surfacant_Mineralization(
                volume = sample_volume, components_dict = components_dict,volf_dict = volf_dict, 
                molarity_dict = molarity_dict, stock_dict = stock_dict)
            new_sample.transfer_ml_to_ul()
            for transfer in new_sample.converted_transfer_dict:
                if (new_sample.converted_transfer_dict[transfer] < 5.0) and new_sample.converted_transfer_dict[transfer] != .0:
#                     print("Rejected Sample:")
#                     print(new_sample.converted_transfer_dict[transfer], transfer)
                    rejected_sample_object_tuple_list.append(({'molarity_dict':molarity_dict,'volf_dict': volf_dict}, "Below pipetting volume", transfer))
                    sample_ok = False
#                     print("\n")

            if sample_ok:  
#                 print("Accepted Sample:")
                volume = 0.0
                for transfer in new_sample.assigned_volumes_dict:
                    volume+=new_sample.assigned_volumes_dict[transfer]
#                 print("Sample volume =", volume)
                safe_sample_object_list.append(new_sample)
#                 print("\n")
            else:
                pass

            
            
    return safe_sample_object_list, rejected_sample_object_tuple_list

def make_rejected_sample(sample_volume, components_dict, volf_dict, molarity_dict, stock_dict):
    sample_ok = True
    
    new_sample = Oil_Surfacant_Mineralization(
                volume = sample_volume, components_dict = components_dict,volf_dict = volf_dict, 
                molarity_dict = molarity_dict, stock_dict = stock_dict)
    new_sample.transfer_ml_to_ul()
    
    for transfer in new_sample.converted_transfer_dict:
                if (new_sample.converted_transfer_dict[transfer] < 5.0) and new_sample.converted_transfer_dict[transfer] != .0:
#                     print("Rejected Sample:")
#                     print(new_sample.converted_transfer_dict[transfer], transfer)
#                     print("\n")
                    sample_ok = False
                    break
      
    if sample_ok:
#         print("Accepted Sample:")
        volume = 0.0
        for transfer in new_sample.assigned_volumes_dict:
            volume+=new_sample.assigned_volumes_dict[transfer]
#         print("Sample volume =", volume)
#         print("\n")
    
    return new_sample, sample_ok



In [30]:
sample_volume = 1

#sample_stock_dict_list is a list of all samples to be made, each object is a dictionary:
#e.g. sample_stock_dict_list[n]= {'sample':sample object, 'stock_option':stock_dict}
sample_stock_dict_list= []

#Sample component definition
components_dict = {'stabilizer1':sodiumdodecylsulfate, 'solvent1':water, 
                   'hydrophobe':hexadecane, 'precursor':HAuCl4, 'buffer': hepes}

#Stock instantiation variables
stabilizer1_stock = A1Stock(
    components_dict = {'A':sodiumdodecylsulfate, 'solvent1':water}, 
    wtf_dict = {'A':.001, 'solvent1':(1.0-.001)})

hydrophobe_stock = A1Stock(
    components_dict = {'A':hexadecane, 'solvent1':water}, 
    wtf_dict = {'A':.0005, 'solvent1':(1.0-.0005)})

precursor_stock = A1Stock(
    components_dict = {'A':HAuCl4, 'solvent1':water}, 
    wtf_dict ={'A':.001, 'solvent1':(1.0-.001)})

buffer_stock = A1Stock(
    components_dict = {'A':hepes, 'solvent1':water}, 
    wtf_dict ={'A':.001, 'solvent1':(1.0-.001)})

stock_dict  = {'stabilizer1':stabilizer1_stock, 'hydrophobe':hydrophobe_stock, 
              'precursor':precursor_stock, 'buffer': buffer_stock}

dilute_stabilizer1_stock = A1Stock(
    components_dict = {'A':sodiumdodecylsulfate, 'solvent1':water}, 
    wtf_dict = {'A':.0001, 'solvent1':(1.0-.0001)})

dilute_buffer_stock = A1Stock(
    components_dict = {'A':hepes, 'solvent1':water}, 
    wtf_dict = {'A':.0001, 'solvent1':(1.0-.0001)})

dilute_stock_dict1 = {'stabilizer1':dilute_stabilizer1_stock, 'hydrophobe':hydrophobe_stock, 
              'precursor':precursor_stock, 'buffer': buffer_stock}

dilute_stock_dict2 = {'stabilizer1':stabilizer1_stock, 'hydrophobe':hydrophobe_stock, 
              'precursor':precursor_stock, 'buffer': dilute_buffer_stock}

dilute_stock_dict3 = {'stabilizer1':dilute_stabilizer1_stock, 'hydrophobe':hydrophobe_stock, 
              'precursor':precursor_stock, 'buffer': dilute_buffer_stock}

#Experiment parameter definitions
volf_dict_list = [{'hydrophobe':0, 'solvent1':float(1.0)}]

stabilizer1_molarity_list = np.logspace(-5,-3, num = 3)
precursor_molarity_list = [1e-4,1e-3]
buffer_molarity_list = np.logspace(-5,-3, num = 3)

stabilizer1_molarity_list = np.append(stabilizer1_molarity_list, [0.0]) 
# precursor_molarity_list = np.append(precursor_molarity_list, [0.0]) 
buffer_molarity_list = np.append(buffer_molarity_list, [0.0]) 


component_range_dict = {'stabilizer1':stabilizer1_molarity_list, 
                        'precursor':precursor_molarity_list, 'buffer':buffer_molarity_list}     


for component_key in components_dict:
    if ('hydrophobe' in component_key):
        stock_dict[component_key].wtf_to_volf()
        dilute_stock_dict1[component_key].wtf_to_volf()
        dilute_stock_dict2[component_key].wtf_to_volf()
        dilute_stock_dict3[component_key].wtf_to_volf()


    elif 'solvent1' not in component_key:
        stock_dict[component_key].wtf_to_molarity()
        dilute_stock_dict1[component_key].wtf_to_molarity()
        dilute_stock_dict2[component_key].wtf_to_molarity()
        dilute_stock_dict2[component_key].wtf_to_molarity()


component_molarity_range_dict_list = dict_product(component_range_dict)

#Make majority of samples with make_sample_list function:
sample_list1, rejected_sample_list = make_sample_list(sample_volume, components_dict, volf_dict_list, component_molarity_range_dict_list, stock_dict)
for sample in sample_list1:
    dict_of_tracked_properties = {'sample':sample}
    for stock_key in stock_dict:
        dict_of_tracked_properties[stock_key] = stock_dict[stock_key]
    
    sample_stock_dict_list.append(dict_of_tracked_properties)

print("Accepted samples = ",len(sample_list1),"Rejected samples = ", len(rejected_sample_list))
print("Trying to make rejected samples")

for rejected_sample in rejected_sample_list:
    stock_options = {'dilute_stock_dict1': dilute_stock_dict1,'dilute_stock_dict2': dilute_stock_dict2,
                     'dilute_stock_dict3': dilute_stock_dict3}
    molarity_dict = rejected_sample[0]['molarity_dict']
    volf_dict = rejected_sample[0]['volf_dict']
    remake_boolean = False
    
    for stock_key in stock_options:
        stock_option = stock_options[stock_key]
        remade_sample, remake_boolean = make_rejected_sample(sample_volume, components_dict, volf_dict, molarity_dict, stock_option)
        if remake_boolean == True:
#             print("Success! remade a rejected sample by using:", stock_key)
            break
    else:
        print("Still failed to make sample:", rejected_sample[0])
    dict_of_tracked_properties = {'sample':remade_sample}
    for stock_key in stock_option:
        dict_of_tracked_properties[stock_key] = stock_option[stock_key]
    sample_stock_dict_list.append(dict_of_tracked_properties)


Accepted samples =  18 Rejected samples =  16
Trying to make rejected samples


In [32]:
robot.reset()
tipracks_300 = [labware.load('opentrons-tiprack-300ul',slot) for slot in['11']]
tipracks_50 = [labware.load('opentrons-tiprack-300ul',slot) for slot in['10']]
kwargs_300 = {'mount' : 'left', 'tip_racks' : tipracks_300}
kwargs_50 = {'mount' : 'right', 'tip_racks' : tipracks_50}
p300 = instruments.P300_Single(**kwargs_300)    
p50 = instruments.P50_Single(**kwargs_50)

large_stock = labware.load('opentrons-tuberack-50ml',1)
medium_stock = labware.load('glass-20ml-vial',2)
sample_plate = labware.load('abgene-deep-96-well',5)


water_stock_position1 = large_stock.wells('A1')
water_stock_position2 = large_stock.wells('A2')

stock_position_dict = {
    stabilizer1_stock:medium_stock.wells('A1'),
    dilute_stabilizer1_stock:medium_stock.wells('A2'), 
    buffer_stock:medium_stock.wells('A3'),
    dilute_buffer_stock:medium_stock.wells('A4'),
    precursor_stock:medium_stock.wells('B1'),
    hydrophobe_stock:medium_stock.wells('B2')
}




# Cell for checking how much of each stock is necessary to make.

In [33]:
solvent_sum_dict = {'solvent1':0, 'stabilizer1':0, 'precursor':0, 
                    'hydrophobe':0, 'buffer':0}
    
#OOF this solvent sum is bad. It doesn't distinguish between which stock volume is being taken from 
for sample_dict in sample_stock_dict_list:
    for component_key in sample_dict['sample'].converted_transfer_dict:
        solvent_sum_dict[component_key] += sample_dict['sample'].converted_transfer_dict[component_key]/1000
    
    
    
print(solvent_sum_dict, '\n')


for stock in stock_position_dict:
    print(stock.componentA.name)
    print("weight fraction = ", stock.componentA_wtf)
    print((stock_position_dict[stock]))
    
    print(stock_position_dict[stock].parent.wells)

    if hasattr(stock, 'componentA_molarity'):
        print("molarity = ", stock.componentA_molarity)
    elif hasattr(stock, 'componentA_volf'):
        print("volume fraction = ", stock.componentA_volf)

    print("____________________________________________________________")



{'solvent1': 22.4498792778335, 'stabilizer1': 2.83454924774323, 'precursor': 6.373192577733199, 'hydrophobe': 0.0, 'buffer': 2.3423788966900716} 

sodiumdodecylsulfate
weight fraction =  0.001
<Well A1>
<bound method Container.wells of <Deck><Slot 2><Container glass-20ml-vial>>
molarity =  0.003457339824948331
____________________________________________________________
sodiumdodecylsulfate
weight fraction =  0.0001
<Well A2>
<bound method Container.wells of <Deck><Slot 2><Container glass-20ml-vial>>
molarity =  0.0003457339824948331
____________________________________________________________
hepes
weight fraction =  0.001
<Well A3>
<bound method Container.wells of <Deck><Slot 2><Container glass-20ml-vial>>
molarity =  0.004183780862202961
____________________________________________________________
hepes
weight fraction =  0.0001
<Well A4>
<bound method Container.wells of <Deck><Slot 2><Container glass-20ml-vial>>
molarity =  0.00041837808622029605
___________________________________

In [6]:
# # print(sample_stock_dict_list[0])
# for sample in sample_stock_dict_list:
#     for key in sample:
#         print(key, sample_stock_dict_list[0][key], '\n')
# #         if(sample_stock_dict_list[0][key] in stock_position_dict):
            



    
def pipette_stock_to_well(component_key, stock_position_dict, sample_stock_dict_list, destination_offset = 0, mix = False):
    pipette = p300
    for sample_dict in sample_stock_dict_list:
        if 'solvent' in component_key:
            source = water_stock_position1
            volume = sample_dict['sample'].converted_transfer_dict[component_key]
            destination = sample_plate[sample_stock_dict_list.index(sample_dict)+destination_offset]
        else:
            stock_used = sample_dict[component_key]
            source = stock_position_dict[stock_used]
            volume = sample_dict['sample'].converted_transfer_dict[component_key]
            destination = sample_plate[sample_stock_dict_list.index(sample_dict)+destination_offset]
        print("Transferring ", volume, "from ", source, "to ", destination)
    
        if volume >p50.max_volume:
            if pipette == p50:
                if not pipette.tip_attached: pipette.drop_tip()
            pipette = p300
        else:
            if pipette == p300:
                if not pipette.tip_attached: pipette.drop_tip()
            pipette = p50
        if not pipette.tip_attached:
            pipette.pick_up_tip()
        
        if sample_stock_dict_list.index(sample_dict) == 0:
            pipette.mix(3, pipette.max_volume, source)
            
        pipette.set_flow_rate(dispense = 1000)
        
        pipette.transfer(volume, source, destination.top(), new_tip ='never' )
        
        
        pipette.blow_out()
        if mix == True:
            pipette.mix(3, pipette.max_volume, destination)
            if not pipette.tip_attached: print("pipette is dropped after mix")
            pipette.drop_tip()
            
    if pipette.tip_attached:
        pipette.drop_tip()
            
        
    return

In [ ]:
pipette_stock_to_well('solvent1', stock_position_dict, sample_stock_dict_list, destination_offset= 40)
pipette_stock_to_well('buffer', stock_position_dict, sample_stock_dict_list)
pipette_stock_to_well('stabilizer1', stock_position_dict, sample_stock_dict_list)

In [35]:
pipette_stock_to_well('hydrophobe', stock_position_dict, sample_stock_dict_list, mix = True)

Transferring  0.0 from  <Well B2> to  <Well A1>
Transferring  0.0 from  <Well B2> to  <Well B1>
Transferring  0.0 from  <Well B2> to  <Well C1>
Transferring  0.0 from  <Well B2> to  <Well D1>
Transferring  0.0 from  <Well B2> to  <Well E1>
Transferring  0.0 from  <Well B2> to  <Well F1>
Transferring  0.0 from  <Well B2> to  <Well G1>
Transferring  0.0 from  <Well B2> to  <Well H1>
Transferring  0.0 from  <Well B2> to  <Well A2>
Transferring  0.0 from  <Well B2> to  <Well B2>
Transferring  0.0 from  <Well B2> to  <Well C2>
Transferring  0.0 from  <Well B2> to  <Well D2>
Transferring  0.0 from  <Well B2> to  <Well E2>
Transferring  0.0 from  <Well B2> to  <Well F2>
Transferring  0.0 from  <Well B2> to  <Well G2>
Transferring  0.0 from  <Well B2> to  <Well H2>
Transferring  0.0 from  <Well B2> to  <Well A3>
Transferring  0.0 from  <Well B2> to  <Well B3>
Transferring  0.0 from  <Well B2> to  <Well C3>
Transferring  0.0 from  <Well B2> to  <Well D3>
Transferring  0.0 from  <Well B2> to  <W

In [8]:
pipette_stock_to_well('precursor', stock_position_dict, sample_stock_dict_list, mix = True)

Cannot drop tip without a tip attached.


Transferring  34.08124373119358 from  <Well B1> to  <Well A1>
Transferring  34.08124373119358 from  <Well B1> to  <Well B1>
Transferring  34.08124373119358 from  <Well B1> to  <Well C1>


Cannot drop tip without a tip attached.


Transferring  340.8124373119358 from  <Well B1> to  <Well D1>
Transferring  340.8124373119358 from  <Well B1> to  <Well E1>
Transferring  340.8124373119358 from  <Well B1> to  <Well F1>


Cannot drop tip without a tip attached.


...

 34.08124373119358 from  <Well B1> to  <Well E4>
Transferring  340.8124373119358 from  <Well B1> to  <Well F4>
Transferring 

Cannot drop tip without a tip attached.
Cannot drop tip without a tip attached.


 34.08124373119358 from  <Well B1> to  <Well G4>
Transferring  340.8124373119358 from  <Well B1> to  <Well H4>


Cannot drop tip without a tip attached.
Cannot drop tip without a tip attached.


Transferring  34.08124373119358 from  <Well B1> to  <Well A5>
Transferring  340.8124373119358 from  <Well B1> to  <Well B5>


In [9]:
for c in robot.commands():
    print(c)

Picking up tip from well A1 in "11"
Mixing 3 times with a volume of 300.0 ul
Aspirating 300.0 uL from well A1 in "1" at 1.0 speed
Dispensing 300.0 uL into well A1 in "1" at 1.0 speed
Aspirating 300.0 uL from well A1 in "1" at 1.0 speed
Dispensing 300.0 uL into well A1 in "1" at 1.0 speed
Aspirating 300.0 uL from well A1 in "1" at 1.0 speed
Dispensing 300.0 uL into well A1 in "1" at 1.0 speed
Transferring 912.8928941039838 from well A1 in "1" to well A1 in "5"
Aspirating 300.0 uL from well A1 in "1" at 1 speed
Dispensing 300.0 uL into well A1 in "5" at 1 speed
Aspirating 300.0 uL from well A1 in "1" at 1 speed
Dispensing 300.0 uL into well A1 in "5" at 1 speed
Aspirating 156.44644705199192 uL from well A1 in "1" at 1 speed
Dispensing 156.44644705199192 uL into well A1 in "5" at 1 speed
Aspirating 156.44644705199192 uL from well A1 in "1" at 1 speed
Dispensing 156.44644705199192 uL into well A1 in "5" at 1 speed
Blowing out
Transferring 697.7764648161203 from well A1 in "1" to well B1 in

Dropping tip into well A1 in "12"
Picking up tip from well E2 in "10"
Transferring 34.08124373119358 from well B1 in "2" to well D3 in "5"
Aspirating 34.08124373119358 uL from well B1 in "2" at 1 speed
Dispensing 34.08124373119358 uL into well D3 in "5" at 1 speed
Blowing out
Mixing 3 times with a volume of 50.0 ul
Aspirating 50.0 uL from well D3 in "5" at 1.0 speed
Dispensing 50.0 uL into well D3 in "5" at 1.0 speed
Aspirating 50.0 uL from well D3 in "5" at 1.0 speed
Dispensing 50.0 uL into well D3 in "5" at 1.0 speed
Aspirating 50.0 uL from well D3 in "5" at 1.0 speed
Dispensing 50.0 uL into well D3 in "5" at 1.0 speed
Dropping tip into well A1 in "12"
Picking up tip from well F2 in "10"
Transferring 34.08124373119358 from well B1 in "2" to well E3 in "5"
Aspirating 34.08124373119358 uL from well B1 in "2" at 1 speed
Dispensing 34.08124373119358 uL into well E3 in "5" at 1 speed
Blowing out
Mixing 3 times with a volume of 50.0 ul
Aspirating 50.0 uL from well E3 in "5" at 1.0 speed
Di